# Klasifikasi teks dengan scikit-learn
**Galuh (galuh.tunggadewi@gmail.com)**

Di *workshop* ini, kita akan belajar gimana sih caranya membuat sebuah sistem klasifikasi teks dengan menggunakan scikit-learn.

## Instalasi

In [ ]:
#@title {display-mode: "form"} 
%%capture
!pip install PySastrawi

In [ ]:
#@title {display-mode: "form"} 

from sklearn.metrics import confusion_matrix, classification_report 
import seaborn as sns 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None) # set column width to maximum so we can see the entire text
import string
from string import digits
import re

from stopwords import STOPWORDS
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
df_train = pd.read_csv('train (1).csv')
df_test = pd.read_csv('test (1).csv')

In [ ]:
df_train

,text,context,intent,sentiment
0,agak miris masih banyak yg ga mau di vaksin terlebih malah keluarga sendiri <URL>,1.0,3.0,1.0
1,malu banget sholat ied pake ada acara mau pingsan segala takut di sangka covid,3.0,4.0,1.0
2,<USERNAME> <USERNAME> <USERNAME> <USERNAME> <USERNAME> memang ga kena covid pun bisa mati tp ya ikhtiar supaya ga mempersulit diri sendiri dan org sekitar karena kalo wabahnya tersebar ya zolim kan hadist nabi pun kalo ada wabah di wilayahmu jgn pergi diamlah di tmptmu,2.0,0.0,1.0
3,terus semangat jaga jarak dan pakai masker karena covid <NUM> masih ada di sekitar kita ya,3.0,4.0,2.0
4,jadi wadm gak lulus ya ginii <URL>,3.0,4.0,2.0
...,...,...,...,...
9035,<USERNAME> eh penghianat tukang gosok gak usah ikutan komen ya gak ada urusanmu dengan soal ini vaksin gratis tetap jalan yang mau cepat dan mampu bayar lebih silakan vaksin mandiri udah kekurangan gorengan ya sana bilang bohir loe,1.0,1.0,1.0
9036,ibuk mbak mau ibuk masakin apa walau udah punya kerjaan dan bertahun tahun merantau masih aja pertanyaannya sama kayak jaman asrama di masa sekolah i will always be her lil baby no matter how old i am mom i miss you harusnya flight bisa wfh di rumah tapi ppkm ini lho,2.0,0.0,1.0
9037,<USERNAME> proning kalau megap megap hypoxia d dimer tinggi pake terapi oksigen hiperbarik hbot tetap tenang <NUM> <NUM> <NUM> pasien corona sembuh pake ngga pake susu beruang ivermectin dll yg tidak bermanfaat sembuh <URL>,3.0,3.0,2.0
9038,"@iimfahima Setuju sekali kak, sedih kadang ketika melihat orang2 terdekat yg katanya beriman tapi masih gak percaya covid yg jelas bisa dibuktikan sungguh sangat meresahkan, g tau lagi harus bagaimana menghadapinya",2.0,0.0,0.0


## Persiapan input

In [ ]:
REGEX_URL = re.compile(
    r"(?:^|(?<![\w\/\.]))"
    # protocol identifier
    # r"(?:(?:https?|ftp)://)"
    r"(?:(?:https?:\/\/|ftp:\/\/|www\d{0,3}\.))"
    # user:pass authentication
    r"(?:\S+(?::\S*)?@)?" r"(?:"
    # IP address exclusion
    # private & local networks
    r"(?!(?:10|127)(?:\.\d{1,3}){3})"
    r"(?!(?:169\.254|192\.168)(?:\.\d{1,3}){2})"
    r"(?!172\.(?:1[6-9]|2\d|3[0-1])(?:\.\d{1,3}){2})"
    # IP address dotted notation octets
    # excludes loopback network 0.0.0.0
    # excludes reserved space >= 224.0.0.0
    # excludes network & broadcast addresses
    # (first & last IP address of each class)
    r"(?:[1-9]\d?|1\d\d|2[01]\d|22[0-3])"
    r"(?:\.(?:1?\d{1,2}|2[0-4]\d|25[0-5])){2}"
    r"(?:\.(?:[1-9]\d?|1\d\d|2[0-4]\d|25[0-4]))"
    r"|"
    # host name
    r"(?:(?:[a-z\\u00a1-\\uffff0-9]-?)*[a-z\\u00a1-\\uffff0-9]+)"
    # domain name
    r"(?:\.(?:[a-z\\u00a1-\\uffff0-9]-?)*[a-z\\u00a1-\\uffff0-9]+)*"
    # TLD identifier
    r"(?:\.(?:[a-z\\u00a1-\\uffff]{2,}))" r"|" r"(?:(localhost))" r")"
    # port number
    r"(?::\d{2,5})?"
    # resource path
    r"(?:\/[^\)\]\}\s]*)?",
    # r"(?:$|(?![\w?!+&\/\)]))",
    flags=re.UNICODE | re.IGNORECASE,
)

REGEX_NUMBER =re.compile(
    r"(?:^|(?<=[^\w,.]))[+–-]?(([1-9]\d{0,2}(,\d{3})+(\.\d*)?)|([1-9]\d{0,2}([ .]\d{3})+(,\d*)?)|(\d*?[.,]\d+)|\d+)(?:$|(?=\b))"
)

def replace(text, regex_pattern, replacement):
  return regex_pattern.sub(replacement, text)

def replace_urls(text, symbol = "<URL>"):
  return replace(text, REGEX_URL, symbol)

def replace_numbers(text, symbol = "<NUM>") -> str:
  return replace(text, REGEX_NUMBER, symbol)

def clean_numbers(text):
  return text.translate({ord(k): None for k in digits})

def replace_punctuations(
        text, symbol = "<PUNCT>", exceptions= []
    ):
    all_punct = [punct for punct in string.punctuation]
    punct_to_remove = "".join(
        [punct for punct in all_punct if punct not in exceptions]
    )
    result = re.sub(r"[" + str(punct_to_remove) + "]+\ *", " " + symbol + " ", text)

    return result

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in STOPWORDS])

def replace_words_by_dictionary(text, dictionary):
    new_text = []
    for word in text.split():
        if word in dictionary:
            new_text.append(dictionary[word])
        else:
            new_text.append(word)

    return " ".join(new_text)

def stem(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()

  output   = stemmer.stem(text)

  return output

In [ ]:
df_slang = pd.read_csv("slang.csv")

In [ ]:
dict_slang = df_slang.set_index('word_from')['word_to'].to_dict()

In [ ]:
def preprocess(text):

  # change slang
  text = replace_words_by_dictionary(text, dict_slang)

  # remove stopwords
  text = remove_stopwords(text)

  # stem
  text = stem(text)

  return text

In [ ]:
def preprocess_final(text):
  # Lowercase
  text = text.lower()

  # Replace URL with tag
  text = replace_urls(text)

  # Remove punctuation
  text = replace_punctuations(text,  symbol="", exceptions=["<", ">"])

  # change slang
  text = replace_words_by_dictionary(text, dict_slang)

  # remove stopwords
  text = remove_stopwords(text)

  # stem
  text = stem(text)

  # Replace numbers with tag
  text = replace_numbers(text)

  text = clean_numbers(text)

  return text

In [ ]:
text = "cari yg seken, murah meriah kualitasnya maS 237329 125rb www.google.com"
preprocess(text)

'cari ken murah riah kualitas mas 237329 125rb www google com'

In [ ]:
df_train["Teks_Clean"] = df_train["text"].apply(preprocess)

In [ ]:
df_train

,text,context,intent,sentiment,Teks_Clean
0,agak miris masih banyak yg ga mau di vaksin terlebih malah keluarga sendiri <URL>,1.0,3.0,1.0,miris vaksin keluarga url
1,malu banget sholat ied pake ada acara mau pingsan segala takut di sangka covid,3.0,4.0,1.0,malu banget sholat ied pakai acara pingsan takut sangka covid
2,<USERNAME> <USERNAME> <USERNAME> <USERNAME> <USERNAME> memang ga kena covid pun bisa mati tp ya ikhtiar supaya ga mempersulit diri sendiri dan org sekitar karena kalo wabahnya tersebar ya zolim kan hadist nabi pun kalo ada wabah di wilayahmu jgn pergi diamlah di tmptmu,2.0,0.0,1.0,username username username username username kena covid mati ya ikhtiar sulit orang kalo wabah sebar ya zolim hadist nabi kalo wabah wilayah pergi diam tmptmu
3,terus semangat jaga jarak dan pakai masker karena covid <NUM> masih ada di sekitar kita ya,3.0,4.0,2.0,semangat jaga jarak pakai masker covid num ya
4,jadi wadm gak lulus ya ginii <URL>,3.0,4.0,2.0,wadm lulus ya ginii url
...,...,...,...,...,...
9035,<USERNAME> eh penghianat tukang gosok gak usah ikutan komen ya gak ada urusanmu dengan soal ini vaksin gratis tetap jalan yang mau cepat dan mampu bayar lebih silakan vaksin mandiri udah kekurangan gorengan ya sana bilang bohir loe,1.0,1.0,1.0,username eh penghianat tukang gosok ikut komen ya urus vaksin gratis jalan cepat bayar sila vaksin mandiri udah kurang goreng ya bilang bohir loe
9036,ibuk mbak mau ibuk masakin apa walau udah punya kerjaan dan bertahun tahun merantau masih aja pertanyaannya sama kayak jaman asrama di masa sekolah i will always be her lil baby no matter how old i am mom i miss you harusnya flight bisa wfh di rumah tapi ppkm ini lho,2.0,0.0,1.0,ibuk mbak ibuk masakin udah kerja tahun rantau aja tanya kayak jaman asrama sekolah i will always be her lil baby no matter how old i am mom i miss you flight wfh rumah ppkm lho
9037,<USERNAME> proning kalau megap megap hypoxia d dimer tinggi pake terapi oksigen hiperbarik hbot tetap tenang <NUM> <NUM> <NUM> pasien corona sembuh pake ngga pake susu beruang ivermectin dll yg tidak bermanfaat sembuh <URL>,3.0,3.0,2.0,username proning megap megap hypoxia dimer pakai terapi oksigen hiperbarik hbot tenang num num num pasien corona sembuh pakai ngga pakai susu beruang ivermectin dll manfaat sembuh url
9038,"@iimfahima Setuju sekali kak, sedih kadang ketika melihat orang2 terdekat yg katanya beriman tapi masih gak percaya covid yg jelas bisa dibuktikan sungguh sangat meresahkan, g tau lagi harus bagaimana menghadapinya",2.0,0.0,0.0,iimfahima tuju kak sedih kadang orang2 dekat iman percaya covid bukti sungguh resah tau hadap


In [ ]:
df_test.sample(5)

,text,context,intent,sentiment
226,cecebs kalo lapar nyerangnya ke yang bertanggung jawab ngasih makan selama ppkm ya jangan sradak sruduk ke sesama netizen dan rakyat percuma yg biasa ngasih makan klean lagi gag punya duid klean nyerang juga gag dibayar u know,2.0,1.0,1.0
1103,ppkm perbanyak pergi ke masjid,3.0,4.0,2.0
1850,@AudistaJ Kami informasikan kembali KRL hanya untuk melayani pekerja di sektor esensial dan kritikal. Bagi #RekanCommuters yang bekerja di sektor non esensial dan non kritikal upayakan bekerja dari rumah. Ayo dukung upaya pemerintah ini untuk menekan penyebaran Covid-19. Tks,0.0,3.0,2.0
103,<USERNAME> <NUM> keluarga besar ku sama orang orang yang aku sayang sehat semuaa <NUM> sekolah offline <NUM> pandemi berakhir,3.0,4.0,1.0
1449,"@jilulisme @BuruhYogyakarta @humas_jogja @Jogja24Jam Bisa ditiru nih disurabaya kebetulan jadi panitia,\n1. Disana kuota 3000\n2. Buat link pendaftaran untuk google form.\n3. Kita 3 hari vaksin, perhari 1000\n4. Kita punya tim 20 untuk balesin WA yg sudah masuk Google form. Jika udah 3000 link kita tutup",1.0,2.0,0.0


# Train

In [ ]:
df_train.head()

,text,context,intent,sentiment,Teks_Clean
0,agak miris masih banyak yg ga mau di vaksin terlebih malah keluarga sendiri <URL>,1.0,3.0,1.0,miris vaksin keluarga url
1,malu banget sholat ied pake ada acara mau pingsan segala takut di sangka covid,3.0,4.0,1.0,malu banget sholat ied pakai acara pingsan takut sangka covid
2,<USERNAME> <USERNAME> <USERNAME> <USERNAME> <USERNAME> memang ga kena covid pun bisa mati tp ya ikhtiar supaya ga mempersulit diri sendiri dan org sekitar karena kalo wabahnya tersebar ya zolim kan hadist nabi pun kalo ada wabah di wilayahmu jgn pergi diamlah di tmptmu,2.0,0.0,1.0,username username username username username kena covid mati ya ikhtiar sulit orang kalo wabah sebar ya zolim hadist nabi kalo wabah wilayah pergi diam tmptmu
3,terus semangat jaga jarak dan pakai masker karena covid <NUM> masih ada di sekitar kita ya,3.0,4.0,2.0,semangat jaga jarak pakai masker covid num ya
4,jadi wadm gak lulus ya ginii <URL>,3.0,4.0,2.0,wadm lulus ya ginii url


In [ ]:
text = df_train["text"]
context = df_train["context"]
sentiment = df_train['sentiment']
intent = df_train['intent']

In [ ]:
text

0                                                                                                                                                                                                        agak miris masih banyak yg ga mau di vaksin  terlebih malah keluarga sendiri   <URL>
1                                                                                                                                                                                                            malu banget sholat ied pake ada acara mau pingsan segala  takut di sangka covid 
2              <USERNAME> <USERNAME> <USERNAME> <USERNAME> <USERNAME> memang ga kena covid pun bisa mati tp ya ikhtiar supaya ga mempersulit diri sendiri dan org sekitar karena kalo wabahnya tersebar ya zolim kan  hadist nabi pun kalo ada wabah di wilayahmu jgn pergi diamlah di tmptmu
3                                                                                                                                             

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline

text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-5, random_state=42,
                          max_iter=5, tol=None)),
])

In [ ]:
text.shape[0]

9040

In [ ]:
context.shape[0]

9040

In [ ]:
text_clf.fit(text, context)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=1e-05, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

## CONTEXT

In [ ]:
text_test = df_test['text']
context_test = df_test['context']

In [ ]:
predicted = text_clf.predict(text_test)
np.mean(predicted == context_test)

0.7712389380530974

In [ ]:
CLASSES=["Policies", "Vaccine", "Life", "Random"]

In [ ]:
from sklearn import metrics
print(metrics.classification_report(context_test, predicted,
    target_names=CLASSES))

              precision    recall  f1-score   support

    Policies       0.58      0.53      0.55       298
     Vaccine       0.94      0.95      0.95       577
        Life       0.65      0.62      0.63       419
      Random       0.77      0.81      0.79       966

    accuracy                           0.77      2260
   macro avg       0.74      0.73      0.73      2260
weighted avg       0.77      0.77      0.77      2260



## Intent

In [ ]:
text_clf_intent = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-5, random_state=68,
                          max_iter=5, tol=None)),
])

text_clf_intent.fit(text, intent)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=1e-05, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [ ]:
text_test = df_test['text']
intent_test = df_test['intent']

In [ ]:
predicted_intent = text_clf_intent.predict(text_test)
np.mean(predicted_intent == intent_test)

0.6668141592920354

In [ ]:
CLASSES_intent=["Opinion", "Complaint", "Suggestion", "News", "Random","Inquiries"]

In [ ]:
from sklearn import metrics
print(metrics.classification_report(intent_test, predicted_intent,
    target_names=CLASSES_intent))

              precision    recall  f1-score   support

     Opinion       0.48      0.57      0.52       289
   Complaint       0.43      0.33      0.37       220
  Suggestion       0.29      0.19      0.23        42
        News       0.75      0.74      0.74       822
      Random       0.72      0.77      0.74       856
   Inquiries       0.33      0.06      0.11        31

    accuracy                           0.67      2260
   macro avg       0.50      0.44      0.45      2260
weighted avg       0.66      0.67      0.66      2260



## Sentiment


In [ ]:
text_clf_sentiment = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                          alpha=1e-5, random_state=68,
                          max_iter=5, tol=None)),
])

text_clf_sentiment.fit(text, sentiment)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=Non...
                ('clf',
                 SGDClassifier(alpha=1e-05, average=False, class_weight=None,
                               early_stopping=False, epsilon=0.1, eta0=0.0,
                               fit_intercept=True, l1_ratio=0.15,
                               learning_rate='optimal', loss='hinge',
                      

In [ ]:
text_test = df_test['text']
sentiment_test = df_test['sentiment']

In [ ]:
predicted_sentiment = text_clf_sentiment.predict(text_test)
np.mean(predicted_sentiment == sentiment_test)

0.7353982300884956

In [ ]:
CLASSES_sentiment=["Positif","Negatif","Netral"]

In [ ]:
from sklearn import metrics
print(metrics.classification_report(sentiment_test, predicted_sentiment,
    target_names=CLASSES_sentiment))

              precision    recall  f1-score   support

     Positif       0.51      0.45      0.48       207
     Negatif       0.70      0.74      0.72       776
      Netral       0.79      0.78      0.79      1277

    accuracy                           0.74      2260
   macro avg       0.67      0.66      0.66      2260
weighted avg       0.73      0.74      0.73      2260



## Membuat prediksi

In [ ]:
def predict_context(message):
  message_clean = preprocess_final(message)
  predicted_label = text_clf.predict([message_clean]).astype(int)
  return message, CLASSES[predicted_label[0]]

def predict_sentiment(message):
  message_clean = preprocess_final(message)
  predicted_label = text_clf_sentiment.predict([message_clean]).astype(int)
  return message, CLASSES_sentiment[predicted_label[0]]

def predict_intent(message):
  message_clean = preprocess_final(message)
  predicted_label = text_clf_intent.predict([message_clean]).astype(int)
  return message, CLASSES_intent[predicted_label[0]]

In [ ]:
predict_context("PPKM diperpanjang, daerah Depok melakukan perpanjangan masa PPKM dan akan menutup semua restoran dan tempat wisata lainnya")

('PPKM diperpanjang, daerah Depok melakukan perpanjangan masa PPKM dan akan menutup semua restoran dan tempat wisata lainnya',
 'Policies')

In [ ]:
predict_intent("PPKM diperpanjang, daerah Depok melakukan perpanjangan masa PPKM dan akan menutup semua restoran dan tempat wisata lainnya")

('PPKM diperpanjang, daerah Depok melakukan perpanjangan masa PPKM dan akan menutup semua restoran dan tempat wisata lainnya',
 'News')

In [ ]:
predict_sentiment("PPKM diperpanjang, daerah Depok melakukan perpanjangan masa PPKM dan akan menutup semua restoran dan tempat wisata lainnya")

('PPKM diperpanjang, daerah Depok melakukan perpanjangan masa PPKM dan akan menutup semua restoran dan tempat wisata lainnya',
 'Netral')

In [ ]:
predict_context("WOI PPKM KAYAK ANJING TAUGA, gw udah bosen dirumah, udah PSBB sekarang malah PPKM anjir, kayak Kontol cape gw di rumah terus mau main bareng temenlah")

('WOI PPKM KAYAK ANJING TAUGA, gw udah bosen dirumah, udah PSBB sekarang malah PPKM anjir, kayak Kontol cape gw di rumah terus mau main bareng temenlah',
 'Life')

In [ ]:
predict_intent("WOI PPKM KAYAK ANJING TAUGA, gw udah bosen dirumah, udah PSBB sekarang malah PPKM anjir, kayak Kontol cape gw di rumah terus mau main bareng temenlah")

('WOI PPKM KAYAK ANJING TAUGA, gw udah bosen dirumah, udah PSBB sekarang malah PPKM anjir, kayak Kontol cape gw di rumah terus mau main bareng temenlah',
 'Complaint')

In [ ]:
predict_sentiment("WOI PPKM KAYAK ANJING TAUGA, gw udah bosen dirumah, udah PSBB sekarang malah PPKM anjir, kayak Kontol cape gw di rumah terus mau main bareng temenlah")

('WOI PPKM KAYAK ANJING TAUGA, gw udah bosen dirumah, udah PSBB sekarang malah PPKM anjir, kayak Kontol cape gw di rumah terus mau main bareng temenlah',
 'Netral')